In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

os.chdir("/content/drive/My Drive/Notebooks/PAPER_EXPERIMENT")

In [3]:
import pandas as pd
from gensim.models import word2vec
import numpy as np
import matplotlib.pyplot as plt
import random 

# Load Dataset

In [ ]:
train_reviews = pd.read_csv("MultiLabel_Classification/data/train.csv")
dev_reviews = pd.read_csv("MultiLabel_Classification/data/dev.csv")

# Convert str to list

In [ ]:
train_reviews["review_tokenized"] = train_reviews["review_tokenized"].apply(
    lambda x: eval(x)
)
dev_reviews["review_tokenized"] = dev_reviews["review_tokenized"].apply(
    lambda x: eval(x)
)

# Training Word2Vec

In [ ]:
corpus = train_reviews["review_tokenized"].tolist() + \
            dev_reviews["review_tokenized"].tolist()

In [ ]:
sum(list(map(lambda x: len(x), corpus)))

3905149

In [ ]:
num_iter = 10

model = word2vec.Word2Vec(
    corpus, min_count=1, size=768, iter=num_iter
)

In [ ]:
if not os.path.exists("Word2Vec"):
    os.mkdir("Word2Vec")

model.save("Word2Vec/word2vec.model")

# Load Pretrained Model

In [4]:
model = word2vec.Word2Vec.load("Word2Vec/word2vec.model")

In [ ]:
# Add [PAD], [UNK] token embeddings

embeddings = np.concatenate(
    [np.zeros([2, 768], dtype=np.float32), model.wv.vectors]
)
np.save("Word2Vec/embeddings", embeddings)

In [ ]:
vocab = model.wv.index2word
vocab.insert(0, "[PAD]")
vocab.insert(1, "[UNK]")

with open("Word2Vec/vocab.txt", "w", encoding="utf-8") as fp:
    fp.write("\n".join(vocab))

In [ ]:
# Store label embeddings ["价格", "位置", "服务", "环境", "菜品"]

label_embeddings = model.wv[["价格", "位置", "服务", "环境", "菜品"]]
np.save("Word2Vec/label_embeddings", label_embeddings)

In [ ]:
with open("Word2Vec/label_words.txt", "w", encoding="utf-8") as fp:
    fp.write("\n".join(["价格", "位置", "服务", "环境", "菜品"]))

# Example

In [ ]:
word = "薯条"
model.wv[word]